# Ανάλυση Πελατών E-commerce

Αυτό το notebook περιέχει τον κώδικα για την ανάλυση τμηματοποίησης πελατών (customer segmentation). Η ανάλυση περιλαμβάνει:
1.  Εύρεση των top 10 πελατών βάσει συνολικών εσόδων.
2.  Υπολογισμό του αριθμού μοναδικών κατηγοριών προϊόντων από τις οποίες αγοράζει κάθε πελάτης.

In [1]:
import os
import pandas as pd

# Δες πού βρίσκεσαι
print("Current directory:", os.getcwd())
print("Files here:", os.listdir('.'))

# Φόρτωση δεδομένων από το φάκελο data/
customers = pd.read_csv('../data/customers.csv')
orders = pd.read_csv('../data/orders.csv')
order_details = pd.read_csv('../data/order_details.csv')
products = pd.read_csv('../data/products.csv')

Current directory: C:\Users\ΚΩΣΤΑΣ\desktop\data_analysis\01-customer-segmentation-analysis
Files here: ['.ipynb_checkpoints', '01-customer-segmentation-analysis', 'analysis.ipynb', 'README.md']


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
# Σε ένα νέο κελί στο Jupyter, γράψε και τρέξε:
import sys
!{sys.executable} -m pip install pandas

# Μετά δοκίμασε πάλι το import
import pandas as pd
print("Pandas installed successfully!")

Pandas installed successfully!



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\ΚΩΣΤΑΣ\python\python.exe -m pip install --upgrade pip


In [4]:
# Φόρτωση των αρχείων CSV

customers = pd.read_csv('../data/customers.csv')
orders = pd.read_csv('../data/orders.csv')
order_details = pd.read_csv('../data/order_details.csv')
products = pd.read_csv('../data/products.csv')

print("Δεδομένα φορτώθηκαν επιτυχώς!")
print(f"Πελάτες: {customers.shape}")
print(f"Παραγγελίες: {orders.shape}")
print(f"Λεπτομέρειες: {order_details.shape}")
print(f"Προϊόντα: {products.shape}")

Δεδομένα φορτώθηκαν επιτυχώς!
Πελάτες: (200, 10)
Παραγγελίες: (1000, 9)
Λεπτομέρειες: (2947, 8)
Προϊόντα: (100, 11)


In [5]:
# Συγχώνευση όλων των πινάκων για πλήρη εικόνα
full_data = (
    order_details
    .merge(orders, on='OrderID')
    .merge(customers, on='CustomerID')
    .merge(products, on='ProductID')
)

print(f"Σύνολο εγγραφών: {len(full_data)}")
full_data.head(3)

Σύνολο εγγραφών: 2947


,OrderDetailID,OrderID,ProductID,Ποσότητα,Τιμή_Μονάδας,Έκπτωση_%,Επιστροφή,Βαθμολογία,CustomerID,Ημερομηνία_Παραγγελίας,...,Όνομα_Προϊόντος,Κατηγορία,Υποκατηγορία,Τιμή,Κόστος,Ποσότητα_Απόθεμα,Ελάχιστο_Απόθεμα,Προμηθευτής,Χώρα_Προέλευσης,Εγγύηση_Μήνες
0,1,ORD000001,PROD00060,2,458.77,8.6,False,3.0,CUST00090,2023-01-04,...,Προϊόν_PROD00060,Ρούχα,Ανδρικά,458.77,114.46,177,7,Προμηθευτής_B,Ελλάδα,6
1,2,ORD000001,PROD00074,7,424.53,18.4,True,2.0,CUST00090,2023-01-04,...,Προϊόν_PROD00074,Βιβλία,Ιστορία,424.53,75.30,50,5,Προμηθευτής_B,Κίνα,12
2,3,ORD000001,PROD00006,10,367.61,22.0,False,2.0,CUST00090,2023-01-04,...,Προϊόν_PROD00006,Τρόφιμα,Φρέσκα,367.61,31.59,165,16,Προμηθευτής_C,Γερμανία,12


In [6]:
# Υπολογισμός τελικής αξίας ανά γραμμή παραγγελίας
data_with_value = full_data.assign(
    Συνολική_Αξία = lambda df: df['Ποσότητα'] * df['Τιμή_Μονάδας'],
    Έκπτωση_Ποσό = lambda df: df['Συνολική_Αξία'] * (df['Έκπτωση_%'] / 100),
    Τελική_Αξία = lambda df: df['Συνολική_Αξία'] - df['Έκπτωση_Ποσό']
)

print("Υπολογισμοί ολοκληρώθηκαν")
data_with_value[['OrderID', 'ProductID', 'Ποσότητα', 'Τιμή_Μονάδας', 'Έκπτωση_%', 'Τελική_Αξία']].head(3)

Υπολογισμοί ολοκληρώθηκαν


,OrderID,ProductID,Ποσότητα,Τιμή_Μονάδας,Έκπτωση_%,Τελική_Αξία
0,ORD000001,PROD00060,2,458.77,8.6,838.63156
1,ORD000001,PROD00074,7,424.53,18.4,2424.91536
2,ORD000001,PROD00006,10,367.61,22.0,2867.35800


In [7]:
# Υπολογισμός συνολικών εσόδων ανά πελάτη
customer_revenue = (
    data_with_value
    .groupby(['CustomerID', 'Όνομα'])
    ['Τελική_Αξία']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# Top 10
top_10_customers = customer_revenue.head(10)
print("ΟΙ TOP 10 ΠΕΛΑΤΕΣ ΒΑΣΕΙ ΕΣΟΔΩΝ")
print(top_10_customers.to_string(index=False))

# Υπολογισμός ποσοστού εσόδων
total_revenue = customer_revenue['Τελική_Αξία'].sum()
top_10_revenue = top_10_customers['Τελική_Αξία'].sum()
print(f"\nΣυνολικά έσοδα από top 10: {top_10_revenue:.2f}")
print(f"Ποσοστό επί συνόλου: {top_10_revenue/total_revenue*100:.1f}%")

ΟΙ TOP 10 ΠΕΛΑΤΕΣ ΒΑΣΕΙ ΕΣΟΔΩΝ
CustomerID       Όνομα  Τελική_Αξία
 CUST00118 Πελάτης_118  49302.67619
 CUST00130 Πελάτης_130  45055.82248
 CUST00046  Πελάτης_46  43446.18419
 CUST00083  Πελάτης_83  41811.51646
 CUST00178 Πελάτης_178  38295.66906
 CUST00104 Πελάτης_104  38269.35955
 CUST00138 Πελάτης_138  37669.35183
 CUST00116 Πελάτης_116  37367.91502
 CUST00009   Πελάτης_9  36657.63059
 CUST00192 Πελάτης_192  35743.80606

Συνολικά έσοδα από top 10: 403619.93
Ποσοστό επί συνόλου: 11.3%


In [8]:
# Υπολογισμός μοναδικών κατηγοριών ανά πελάτη
customer_categories = (
    data_with_value
    .groupby(['CustomerID', 'Όνομα'])
    ['Κατηγορία']
    .nunique()
    .reset_index()
    .rename(columns={'Κατηγορία': 'Αριθμός_Κατηγοριών'})
)

print("ΚΑΤΑΝΟΜΗ ΠΕΛΑΤΩΝ ΑΝΑ ΑΡΙΘΜΟ ΚΑΤΗΓΟΡΙΩΝ")
print(customer_categories['Αριθμός_Κατηγοριών'].value_counts().sort_index())

# Πελάτες με >=3 κατηγορίες
customers_3plus = customer_categories[customer_categories['Αριθμός_Κατηγοριών'] >= 3]
print(f"\nΠελάτες με ≥3 κατηγορίες: {len(customers_3plus)} από {len(customer_categories)}")
print(f"Ποσοστό: {len(customers_3plus)/len(customer_categories)*100:.1f}%")

ΚΑΤΑΝΟΜΗ ΠΕΛΑΤΩΝ ΑΝΑ ΑΡΙΘΜΟ ΚΑΤΗΓΟΡΙΩΝ
Αριθμός_Κατηγοριών
2     4
3     8
4    14
5    24
6    50
7    48
8    51
Name: count, dtype: int64

Πελάτες με ≥3 κατηγορίες: 195 από 199
Ποσοστό: 98.0%


In [9]:
# Συνδυασμός revenue και categories για τους top 10
top_10_detailed = (
    top_10_customers
    .merge(customer_categories, on=['CustomerID', 'Όνομα'])
    .sort_values('Τελική_Αξία', ascending=False)
)

print("ΛΕΠΤΟΜΕΡΕΙΕΣ ΓΙΑ ΤΟΥΣ TOP 10 ΠΕΛΑΤΕΣ")
print(top_10_detailed.to_string(index=False))

ΛΕΠΤΟΜΕΡΕΙΕΣ ΓΙΑ ΤΟΥΣ TOP 10 ΠΕΛΑΤΕΣ
CustomerID       Όνομα  Τελική_Αξία  Αριθμός_Κατηγοριών
 CUST00118 Πελάτης_118  49302.67619                   8
 CUST00130 Πελάτης_130  45055.82248                   8
 CUST00046  Πελάτης_46  43446.18419                   8
 CUST00083  Πελάτης_83  41811.51646                   8
 CUST00178 Πελάτης_178  38295.66906                   8
 CUST00104 Πελάτης_104  38269.35955                   7
 CUST00138 Πελάτης_138  37669.35183                   8
 CUST00116 Πελάτης_116  37367.91502                   7
 CUST00009   Πελάτης_9  36657.63059                   8
 CUST00192 Πελάτης_192  35743.80606                   8
